# api practise

In [2]:
import requests
import json
from datetime import datetime, timedelta, date
import time
import sqlite3
import matplotlib.pyplot as plt
import pandas as pd

справочник криптовлют 

In [61]:
top_coins = [
    'Bitcoin',
    'Ethereum',
    'Tether',
    'USD Coin',
    'BNB'
]

In [119]:
df_info.columns

Index(['id_coins', 'price_in_btc', 'price_in_usd', 'price_in_rub', 'date',
       'day', 'insert_dttm'],
      dtype='object')

созадлдем бд на sql3 lite

In [123]:
conn = sqlite3.connect('coins_price')
curs = conn.cursor()
curs.execute('''
    CREATE TABLE IF NOT EXISTS prices (
        id_coins nvarchar(255),
        price_in_btc REAL,
        price_in_usd REAL,
        price_in_rub REAL,
        date date,
        day INTEGER,
        insert_dttm datetime
        )
        ''')
conn.commit()
curs.execute('''
    SELECT * FROM prices ''')
conn.commit()

фиксируем все уже загруженные даные

In [9]:
dates_saved = [
    datetime.strptime(x[0], '%Y-%m-%d %H:%M:%S') for x in curs.fetchall()
]


In [24]:
def get_coins_list() -> dict:
    # return coins in cons gectocons 
    response = requests.get('https://api.coingecko.com/api/v3/coins/list?include_platform=false')
    print(response.status_code)
    print(response)
    return json.loads(response.text)

def get_history(coin:str, date:str) -> dict:
    # return coin hisdtory per date

    history_url = f'https://api.coingecko.com/api/v3/coins/{coin}/history?date={date}'
    respose = requests.get(history_url)

    return json.loads(respose.text)
                

Получем данные по монетам

In [25]:
%%time

coins = get_coins_list()

200
<Response [200]>
CPU times: user 30.6 ms, sys: 6.88 ms, total: 37.4 ms
Wall time: 543 ms


In [39]:
def parse_coins_id(coins_meta: dict) -> list:
    list_id = []
    for row in coins_meta:
        list_coin = (row['id'], row['symbol'], row['name'])
        list_id.append(list_coin)
    return list_id

генерируем df

In [41]:
df_all_coins = pd.DataFrame(parse_coins_id(coins), columns=['id', 'symble', 'name'])
print(df_all_coins.shape)
df_all_coins.head(15)


(14820, 3)


,id,symble,name
0,01coin,zoc,01coin
1,0chain,zcn,Zus
2,0-knowledge-network,0kn,0 Knowledge Network
3,0-mee,ome,O-MEE
4,0vix-protocol,vix,0VIX Protocol
5,0vm,zerovm,0VM
6,0x,zrx,0x Protocol
7,0x0-ai-ai-smart-contract,0x0,0x0.ai: AI Smart Contract
8,0x404,xfour,0x404
9,0x678-landwolf-1933,wolf,Landwolf


In [62]:
df_top_coins = df_all_coins.loc[df_all_coins['name'].isin(top_coins)]
df_top_coins

,id,symble,name
1627,binancecoin,bnb,BNB
1707,bitcoin,btc,Bitcoin
4674,ethereum,eth,Ethereum
12743,tether,usdt,Tether


In [63]:
delta_period = 14

if dates_saved:
    start_date = max(dates_saved) + timedelta(days=1)
else:
    start_date = date.today() - timedelta(days=delta_period)

end_date = date.today() - timedelta(days=1)
daterange = [x.strftime('%d-%m-%Y') for x in pd.date_range(start_date, end_date)]
daterange


['11-06-2024',
 '12-06-2024',
 '13-06-2024',
 '14-06-2024',
 '15-06-2024',
 '16-06-2024',
 '17-06-2024',
 '18-06-2024',
 '19-06-2024',
 '20-06-2024',
 '21-06-2024',
 '22-06-2024',
 '23-06-2024',
 '24-06-2024']

In [95]:
import numpy as np

df_info = pd.DataFrame()
def get_coins_price_set(coinsid: np.ndarray, daterange:list)->list:

    list_hits = []
    for ids in coinsid:
        for i, date in enumerate(daterange):
            try:
                url_get_hist = f'https://api.coingecko.com/api/v3/coins/{ids}/history?date={date}'
                response = requests.get(url_get_hist)
                content = json.loads(response.text)
                price_cur_btc = content['market_data']['current_price']['btc']
                price_cur_usd = content['market_data']['current_price']['usd']
                price_cur_rub = content['market_data']['current_price']['rub']
                
                time.sleep(12)
                list_hits.append((ids, price_cur_btc, price_cur_usd, price_cur_rub, date, i))
                print(price_cur_btc, price_cur_usd, price_cur_rub, date, i)
            except Exception as e:
                print( f'error {e}')
                continue
    return list_hits
#            requests.get
    


In [96]:
list_hits = get_coins_price_set(df_top_coins['id'], daterange)

0.008980886006956573 624.1111865882733 55466.87626491117 11-06-2024 0
0.008935514500652548 601.6892499687187 53701.541137151355 12-06-2024 1
0.00907134956655721 618.8387734028372 55076.122344540054 13-06-2024 2
0.008973310749353985 599.1566520664575 52648.93095893103 14-06-2024 3
0.00913734026154401 603.0538175934126 53857.23665442335 15-06-2024 4
0.009182784229111332 607.8065562169128 54281.69192082721 16-06-2024 5
0.009154032046773083 609.8330309332431 54266.49719934425 17-06-2024 6
0.009088895336343295 603.9317705958817 53583.84996971023 18-06-2024 7
0.009036616788742464 588.3547299332587 50231.717610298925 19-06-2024 8
0.009217464964665325 598.1823995707931 49948.02159850382 20-06-2024 9
0.009049336333039934 586.9059353494202 51273.51812924145 21-06-2024 10
0.009143703815169482 585.8611030285144 52131.495156261044 22-06-2024 11
0.009193273190362367 590.5596486180493 52549.58200557124 23-06-2024 12
0.009125141153394094 577.112305791683 50938.350786847506 24-06-2024 13
1.0 69493.1776

In [101]:
list_hits

[('binancecoin',
  0.008980886006956573,
  624.1111865882733,
  55466.87626491117,
  '11-06-2024',
  0),
 ('binancecoin',
  0.008935514500652548,
  601.6892499687187,
  53701.541137151355,
  '12-06-2024',
  1),
 ('binancecoin',
  0.00907134956655721,
  618.8387734028372,
  55076.122344540054,
  '13-06-2024',
  2),
 ('binancecoin',
  0.008973310749353985,
  599.1566520664575,
  52648.93095893103,
  '14-06-2024',
  3),
 ('binancecoin',
  0.00913734026154401,
  603.0538175934126,
  53857.23665442335,
  '15-06-2024',
  4),
 ('binancecoin',
  0.009182784229111332,
  607.8065562169128,
  54281.69192082721,
  '16-06-2024',
  5),
 ('binancecoin',
  0.009154032046773083,
  609.8330309332431,
  54266.49719934425,
  '17-06-2024',
  6),
 ('binancecoin',
  0.009088895336343295,
  603.9317705958817,
  53583.84996971023,
  '18-06-2024',
  7),
 ('binancecoin',
  0.009036616788742464,
  588.3547299332587,
  50231.717610298925,
  '19-06-2024',
  8),
 ('binancecoin',
  0.009217464964665325,
  598.1823995

In [103]:
df_info = pd.DataFrame(list_hits, columns=['id_coins','price_in_btc', 'price_in_usd', 'price_in_rub', 'date', 'day'])

In [105]:
pd.set_option('display.float_format', str)

In [106]:
df_info

,id_coins,price_in_btc,price_in_usd,price_in_rub,date,day
0,binancecoin,0.008980886006956573,624.1111865882733,55466.87626491117,11-06-2024,0
1,binancecoin,0.008935514500652548,601.6892499687187,53701.541137151355,12-06-2024,1
2,binancecoin,0.00907134956655721,618.8387734028372,55076.122344540054,13-06-2024,2
3,binancecoin,0.008973310749353985,599.1566520664575,52648.93095893103,14-06-2024,3
4,binancecoin,0.00913734026154401,603.0538175934126,53857.23665442335,15-06-2024,4
5,binancecoin,0.009182784229111332,607.8065562169128,54281.69192082721,16-06-2024,5
6,binancecoin,0.009154032046773083,609.8330309332431,54266.49719934425,17-06-2024,6
7,binancecoin,0.009088895336343295,603.9317705958817,53583.84996971023,18-06-2024,7
8,binancecoin,0.009036616788742464,588.3547299332587,50231.717610298925,19-06-2024,8
9,binancecoin,0.009217464964665325,598.1823995707931,49948.02159850382,20-06-2024,9


In [115]:
#df_info['date'] = df_info['date'].apply(lambda x: datetime.strptime(x,'%d-%m-%Y'))
df_info['insert_dttm'] = datetime.now()

set_history_coins = pd.DataFrame([x for x in curs.fetchall()], columns=['coingecko_id', 'price_in_btc', 'price_in_usd', 'price_in_rub', 'date', 'day','insert_dttm'])
set_history_coins['date'] = set_history_coins['date'].apply(lambda x: datetime.strptime(x,'%d-%m-%Y'))

set_all_coins = pd.concat([set_history_coins, df_info])

/tmp/ipykernel_10995/148339979.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  set_all_coins = pd.concat([set_history_coins, df_info])


In [ ]:
curs.execute('''
    CREATE TABLE IF NOT EXISTS prices (
        coingecko_id nvarchar(255),
        price_date date,
        btc REAL,
        usd REAL,
        insert_dttm datetime
        )
        ''')

In [124]:
set_all_coins.price = set_all_coins['date'].astype('datetime64[ns]')
df_info.to_sql('prices', conn, if_exists='append', index=False)

56

In [127]:
conn = sqlite3.connect('coins_price')
curs = conn.cursor()
curs.execute('''
    SELECT * FROM prices ''')
conn.commit()
rows = curs.fetchall()
conn.close()
for row in rows:
   print(row)
    

('binancecoin', 0.008980886006956573, 624.1111865882733, 55466.87626491117, '2024-06-11 00:00:00', 0, '2024-06-25 02:15:00.364264')
('binancecoin', 0.008935514500652548, 601.6892499687187, 53701.541137151355, '2024-06-12 00:00:00', 1, '2024-06-25 02:15:00.364264')
('binancecoin', 0.00907134956655721, 618.8387734028372, 55076.122344540054, '2024-06-13 00:00:00', 2, '2024-06-25 02:15:00.364264')
('binancecoin', 0.008973310749353985, 599.1566520664575, 52648.93095893103, '2024-06-14 00:00:00', 3, '2024-06-25 02:15:00.364264')
('binancecoin', 0.00913734026154401, 603.0538175934126, 53857.23665442335, '2024-06-15 00:00:00', 4, '2024-06-25 02:15:00.364264')
('binancecoin', 0.009182784229111332, 607.8065562169128, 54281.69192082721, '2024-06-16 00:00:00', 5, '2024-06-25 02:15:00.364264')
('binancecoin', 0.009154032046773083, 609.8330309332431, 54266.49719934425, '2024-06-17 00:00:00', 6, '2024-06-25 02:15:00.364264')
('binancecoin', 0.009088895336343295, 603.9317705958817, 53583.84996971023, 